> Tip: Run the shapes_geog materialization first if the table doesn't exist or is stale. A helper cell to run it is included below.

# Materialize Trip and Route Edges
This notebook lets you set variables and run the parameterized BigQuery SQL in `infrastructure/queries/scheduled/` to:
- Reset the `trip_edges` table schema
- Insert `trip_edges` for a specific `route_id` (or by shards)
- Build the aggregated `route_edges` table
 
It uses Application Default Credentials (ADC) or a service account if configured. Ensure you have access to the target project and dataset.

In [1]:
# Install dependencies (optional; uncomment if needed)
# %pip install -q google-cloud-bigquery google-cloud-bigquery-storage python-dotenv pandas pyarrow tqdm
from __future__ import annotations
import os
import json
from datetime import datetime
from pathlib import Path
from typing import Dict, Any
import pandas as pd
from IPython.display import display
from google.cloud import bigquery
from google.oauth2 import service_account
from tqdm import tqdm

In [2]:
# Configure variables here
project_id = os.environ.get("BQ_PROJECT", "regal-dynamo-470908-v9")
dataset_id = os.environ.get("BQ_DATASET", "auckland_data_dev")
location = os.environ.get("BQ_LOCATION", "australia-southeast1")
# Default route to run (can be changed and rerun)
route_id = os.environ.get("ROUTE_ID", "INN-202")
 
print({"project_id": project_id, "dataset_id": dataset_id, "location": location, "route_id": route_id})

{'project_id': 'regal-dynamo-470908-v9', 'dataset_id': 'auckland_data_dev', 'location': 'australia-southeast1', 'route_id': 'INN-202'}


In [3]:
# Create BigQuery client using ADC or service account JSON if provided
credentials_path = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS_JSON", None)
if credentials_path and Path(credentials_path).exists():
    creds = service_account.Credentials.from_service_account_file(credentials_path)
    client = bigquery.Client(project=project_id, credentials=creds)
else:
    client = bigquery.Client(project=project_id)
# Note: BigQuery region is provided per query via run_sql_loc(location=...)
print(f"BigQuery client ready for {project_id} (region set per job: {location})")

BigQuery client ready for regal-dynamo-470908-v9 (region set per job: australia-southeast1)


In [4]:
from string import Template
def load_sql(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def render_sql(sql: str, params: Dict[str, Any]) -> str:
    # Use Template to replace ${var} placeholders present in our SQL files
    return Template(sql).substitute(params)

def run_sql(sql: str, job_config: bigquery.QueryJobConfig | None = None) -> bigquery.table.RowIterator:
    job = client.query(sql, job_config=job_config)
    result = job.result()
    print(f"Job {job.job_id} done. bytes={job.total_bytes_processed:,} cached={job.cache_hit}")
    return result

def run_sql_loc(sql: str) -> bigquery.table.RowIterator:
    # Region-aware query execution (uses the `location` variable)
    job_config = bigquery.QueryJobConfig()
    job = client.query(sql, job_config=job_config, location=location)
    res = job.result()
    print(f"Job {job.job_id} done. location={job.location} bytes={job.total_bytes_processed:,}")
    return res

In [5]:
# Paths to SQL files
root = Path("/Users/martin/Projects/mak-group/gcp-example")
sql_dir = root / "infrastructure" / "queries" / "scheduled"
trip_edges_reset_path = sql_dir / "trip_edges_reset.sql"
trip_edges_insert_by_route_path = sql_dir / "trip_edges_insert_by_route.sql"
route_edges_path = sql_dir / "route_edges.sql"
print(trip_edges_reset_path)
print(trip_edges_insert_by_route_path)
print(route_edges_path)

/Users/martin/Projects/mak-group/gcp-example/infrastructure/queries/scheduled/trip_edges_reset.sql
/Users/martin/Projects/mak-group/gcp-example/infrastructure/queries/scheduled/trip_edges_insert_by_route.sql
/Users/martin/Projects/mak-group/gcp-example/infrastructure/queries/scheduled/route_edges.sql


In [15]:
# Reset trip_edges table (empty schema + clustering)
params = {"project_id": project_id, "dataset_id": dataset_id}
reset_sql = render_sql(load_sql(str(trip_edges_reset_path)), params)
print(reset_sql[:400] + "...\n")
_ = run_sql_loc(reset_sql)

-- Reset (recreate empty) trip_edges table with schema and clustering
-- Run before shard inserts

CREATE OR REPLACE TABLE `regal-dynamo-470908-v9.auckland_data_dev.trip_edges`
CLUSTER BY route_id, edge_id AS
WITH shapes AS (
  SELECT
    shape_id,
    geom,
  ST_BUFFER(geom, 15) AS buf_geom,
  ST_BOUNDINGBOX(ST_BUFFER(geom, 15)) AS env_box
  FROM `regal-dynamo-470908-v9.auckland_data_dev.shapes_g...

Job 465b4578-73f0-4d60-9782-fea36a8ecbcc done. location=australia-southeast1 bytes=0
Job 465b4578-73f0-4d60-9782-fea36a8ecbcc done. location=australia-southeast1 bytes=0


In [7]:
# Insert trip_edges for a single route_id
params = {"project_id": project_id, "dataset_id": dataset_id, "route_id": route_id}
insert_sql = render_sql(load_sql(str(trip_edges_insert_by_route_path)), params)
print(insert_sql[:400] + "...\n")
_ = run_sql_loc(insert_sql)

-- Insert trip_edges for a single route_id shard
-- Variables: project_id, dataset_id, route_id

INSERT INTO `regal-dynamo-470908-v9.auckland_data_dev.trip_edges`
WITH shapes AS (
  SELECT
    shape_id,
    geom,
  ST_BUFFER(geom, 15) AS buf_geom,
  ST_BOUNDINGBOX(ST_BUFFER(geom, 15)) AS env_box
  FROM `regal-dynamo-470908-v9.auckland_data_dev.shapes_geog`
), roads AS (
  SELECT
    edge_id,
    n...



BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/regal-dynamo-470908-v9/queries/8819baad-39f9-4cb2-a7e8-6e2e65a3a566?maxResults=0&location=australia-southeast1&prettyPrint=false: Query exceeded resource limits. This query used 12326 CPU seconds but would charge only 40M Analysis bytes. This exceeds the ratio supported by the on-demand pricing model. Please consider moving this workload to a capacity-based pricing model, which does not have this limit. 12326 CPU seconds were used, and this query must use less than 10200 CPU seconds.

Location: australia-southeast1
Job ID: 8819baad-39f9-4cb2-a7e8-6e2e65a3a566


In [ ]:
# Build route_edges aggregation from trip_edges
params = {"project_id": project_id, "dataset_id": dataset_id}
agg_sql = render_sql(load_sql(str(route_edges_path)), params)
print(agg_sql[:300] + "...\n")
_ = run_sql_loc(agg_sql)

In [ ]:
# Optional: shard mode insert (set shard_mod and run the loop)
shard_mod = 4  # increase if any shard is heavy
params_base = {"project_id": project_id, "dataset_id": dataset_id, "shard_mod": shard_mod}
trip_edges_insert_shard_path = sql_dir / "trip_edges_insert_shard.sql"
sql_template = load_sql(str(trip_edges_insert_shard_path))
for shard in range(shard_mod):
    params = {**params_base, "shard": shard}
    shard_sql = render_sql(sql_template, params)
    print(f"Running shard {shard}/{shard_mod-1}")
    _ = run_sql_loc(shard_sql)

In [ ]:
# Quick verification: counts and sample rows
verify_sql = f"""
SELECT 
  (SELECT COUNT(*) FROM `{project_id}.{dataset_id}.trip_edges`) AS trip_edges_rows,
  (SELECT COUNT(*) FROM `{project_id}.{dataset_id}.route_edges`) AS route_edges_rows;
"""
list_sql = f"SELECT route_id, COUNT(*) AS edges FROM `{project_id}.{dataset_id}.trip_edges` GROUP BY route_id ORDER BY edges DESC LIMIT 10"
for q in (verify_sql, list_sql):
    df = run_sql_loc(q).to_dataframe()
    display(df)

In [ ]:
# Optional: materialize shapes_geog if needed
shapes_sql_path = sql_dir / "shapes_geog.sql"
shapes_sql = render_sql(load_sql(str(shapes_sql_path)), {"project_id": project_id, "dataset_id": dataset_id})
print(shapes_sql[:300] + "...\n")
_ = run_sql_loc(shapes_sql)

In [ ]:
# Recreate BigQuery client with robust credential handling (overrides previous)
from google.auth.exceptions import DefaultCredentialsError
sa_path = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS") or os.environ.get("GOOGLE_APPLICATION_CREDENTIALS_JSON")
creds = None
if sa_path:
    p = Path(sa_path)
    try:
        if p.exists():
            creds = service_account.Credentials.from_service_account_file(str(p))
        else:
            # Try treating env as JSON content
            try:
                info = json.loads(sa_path)
                creds = service_account.Credentials.from_service_account_info(info)
            except json.JSONDecodeError:
                pass
    except Exception as e:
        print(f"Service account credential parse failed: {e}")
try:
    if creds is not None:
        client = bigquery.Client(project=project_id, credentials=creds)
    else:
        client = bigquery.Client(project=project_id)
    print(f"BigQuery client ready for {project_id} (region set per job: {location})")
except DefaultCredentialsError as e:
    raise RuntimeError("No GCP credentials found. Use gcloud auth application-default login or set GOOGLE_APPLICATION_CREDENTIALS.") from e

In [ ]:
# Helper to run a query in a specific location
def run_sql_loc(sql: str) -> bigquery.table.RowIterator:
    job_config = bigquery.QueryJobConfig()
    job = client.query(sql, job_config=job_config, location=location)
    res = job.result()
    print(f"Job {job.job_id} done. location={job.location} bytes={job.total_bytes_processed:,}")
    return res

## Fallback: chunked insert by shape_id
If the single-route insert exceeds on-demand CPU ratio, run the route in smaller chunks by shape_id. This adds a WHERE shape_id IN (...) to each chunk and repeats until all shapes are processed.

In [16]:
# Helpers: fetch shape_ids for a route and run chunked inserts

def fetch_shape_ids_for_route(route_id: str) -> list[str]:
    q = f"""
    SELECT DISTINCT shape_id
    FROM `{project_id}.{dataset_id}.stg_trips`
    WHERE route_id = @route_id
    ORDER BY shape_id
    """
    job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("route_id", "STRING", route_id)])
    rows = client.query(q, job_config=job_config, location=location).result()
    return [r[0] for r in rows]


def render_insert_with_shape_filter(base_sql: str, shape_ids: list[str], route_id: str) -> str:
    # Safely append an AND shape_id IN (...) predicate to the trips CTE WHERE route_id = "..."
    in_list = ", ".join(["'" + s + "'" for s in shape_ids])
    pattern = f'WHERE route_id = "{route_id}"'
    if pattern in base_sql:
        return base_sql.replace(pattern, pattern + f"\n  AND shape_id IN ({in_list})", 1)
    alt_pattern = f"WHERE route_id = '{route_id}'"
    if alt_pattern in base_sql:
        return base_sql.replace(alt_pattern, alt_pattern + f"\n  AND shape_id IN ({in_list})", 1)
    # Fallback: inject just before the end of the trips CTE
    return base_sql.replace("\n)\nSELECT", f"\n  AND shape_id IN ({in_list})\n)\nSELECT", 1)


def ensure_antijoin_once(sql: str, project_id: str, dataset_id: str) -> str:
    # If the base SQL already includes an anti-join, skip; else inject it
    te_phrase = f"LEFT JOIN `{project_id}.{dataset_id}.trip_edges` te"
    if te_phrase in sql or " te.trip_id IS NULL" in sql:
        return sql
    te = f"`{project_id}.{dataset_id}.trip_edges`"
    join_clause = f"\nLEFT JOIN {te} te ON te.trip_id = t.trip_id AND te.edge_id = r.edge_id\nWHERE te.trip_id IS NULL AND "
    return sql.replace("\nWHERE ", join_clause, 1)


def render_insert_with_shape_filter_and_antijoin(base_sql: str, shape_ids: list[str], route_id: str, project_id: str, dataset_id: str) -> str:
    sql = render_insert_with_shape_filter(base_sql, shape_ids, route_id)
    return ensure_antijoin_once(sql, project_id, dataset_id)

In [17]:
# Run chunked insert for the current route_id
one_shape_at_a_time = True  # set True to process 1 shape_id per query
chunk_size = 1 if one_shape_at_a_time else 10  # adjust smaller if still hitting CPU ratio
all_shapes = fetch_shape_ids_for_route(route_id)
print(f"Route {route_id} has {len(all_shapes)} distinct shape_ids")
params = {"project_id": project_id, "dataset_id": dataset_id, "route_id": route_id}
base_sql = render_sql(load_sql(str(trip_edges_insert_by_route_path)), params)
for i in range(0, len(all_shapes), chunk_size):
    batch = all_shapes[i:i+chunk_size]
    chunk_sql = render_insert_with_shape_filter_and_antijoin(base_sql, batch, route_id, project_id, dataset_id)
    print(f"Inserting chunk {i//chunk_size + 1} with {len(batch)} shape(s): {batch}")
    _ = run_sql_loc(chunk_sql)

Route INN-202 has 16 distinct shape_ids
Inserting chunk 1 with 1 shape(s): ['1286-79821-0d6c86f9']
Job fc4ad879-43b7-48fd-9eec-d16b42fc0b8a done. location=australia-southeast1 bytes=37,636,178
Inserting chunk 2 with 1 shape(s): ['1286-79821-2928d84f']
Job fc4ad879-43b7-48fd-9eec-d16b42fc0b8a done. location=australia-southeast1 bytes=37,636,178
Inserting chunk 2 with 1 shape(s): ['1286-79821-2928d84f']
Job 9c362942-389a-4b36-a15d-fb1c056c53b8 done. location=australia-southeast1 bytes=39,924,443
Inserting chunk 3 with 1 shape(s): ['1286-79823-209afed3']
Job 9c362942-389a-4b36-a15d-fb1c056c53b8 done. location=australia-southeast1 bytes=39,924,443
Inserting chunk 3 with 1 shape(s): ['1286-79823-209afed3']
Job cbb7d21c-62bb-461a-a844-90a63a632f26 done. location=australia-southeast1 bytes=42,212,708
Inserting chunk 4 with 1 shape(s): ['1286-79823-4d73cc7f']
Job cbb7d21c-62bb-461a-a844-90a63a632f26 done. location=australia-southeast1 bytes=42,212,708
Inserting chunk 4 with 1 shape(s): ['1286-

KeyboardInterrupt: 

## Diagnostics: sanity-check trip_edges volume
Let’s check whether the high row count is expected:
- Total trip_edges rows vs distinct (trip_id, edge_id) pairs (duplicate check)
- Number of trips for this route
- Edges-per-trip stats (avg/min/max)
- If available, edges-per-shape from `shape_edges` for this route

In [ ]:
# Diagnose shape_edges volume for current route under different thresholds
params = [bigquery.ScalarQueryParameter("route_id", "STRING", route_id)]
diag_sql = f"""
WITH r_shapes AS (
  SELECT DISTINCT shape_id FROM `{project_id}.{dataset_id}.stg_trips` WHERE route_id = @route_id
), counts AS (
  SELECT
    'current' AS variant, COUNT(*) AS rows
  FROM `{project_id}.{dataset_id}.shape_edges`
  WHERE shape_id IN (SELECT shape_id FROM r_shapes)
), alt AS (
  SELECT
    'alt_strict' AS variant, COUNT(*) AS rows
  FROM (
    WITH s AS (
      SELECT shape_id, geom, ST_BUFFER(geom, 6) AS buf_geom
      FROM `{project_id}.{dataset_id}.shapes_geog`
      WHERE shape_id IN (SELECT shape_id FROM r_shapes)
    ), r AS (
      SELECT edge_id, highway, length_m AS edge_length_m, ST_SIMPLIFY(geom, 7) AS simple_geom
      FROM `{project_id}.{dataset_id}.vw_osm_akl_road_links`
      WHERE highway NOT IN ('footway','path','cycleway','steps','track','pedestrian','bridleway','corridor')
    )
    SELECT s.shape_id, r.edge_id,
           ST_LENGTH(ST_INTERSECTION(r.simple_geom, s.buf_geom)) AS overlap_m, r.edge_length_m
    FROM s JOIN r
    ON ST_DWithin(r.simple_geom, s.buf_geom, 0)
  )
  WHERE overlap_m > 40 AND overlap_m / edge_length_m >= 0.15
)
SELECT * FROM counts
UNION ALL
SELECT * FROM alt
"""
job = client.query(diag_sql, job_config=bigquery.QueryJobConfig(query_parameters=params), location=location)
df = job.result().to_dataframe()
display(df)

In [ ]:
# Diagnostics queries for the current route_id
from IPython.display import display

q_overview = f"""
WITH te AS (
  SELECT * FROM `{project_id}.{dataset_id}.trip_edges` WHERE route_id = @route_id
)
SELECT
  (SELECT COUNT(*) FROM te) AS total_rows,
  (SELECT COUNT(*) FROM (SELECT DISTINCT trip_id, edge_id FROM te)) AS distinct_trip_edge_pairs,
  (SELECT COUNT(DISTINCT trip_id) FROM te) AS trips,
  SAFE_DIVIDE((SELECT COUNT(*) FROM te), (SELECT COUNT(DISTINCT trip_id) FROM te)) AS avg_edges_per_trip
"""

q_edges_per_trip_stats = f"""
SELECT
  APPROX_QUANTILES(cnt, 10) AS deciles,
  MIN(cnt) AS min_edges,
  AVG(cnt) AS avg_edges,
  MAX(cnt) AS max_edges
FROM (
  SELECT trip_id, COUNT(*) AS cnt
  FROM `{project_id}.{dataset_id}.trip_edges`
  WHERE route_id = @route_id
  GROUP BY trip_id
)
"""

q_shape_edges_stats = f"""
SELECT
  COUNT(*) AS shape_edge_rows,
  COUNT(DISTINCT shape_id) AS shapes,
  SAFE_DIVIDE(COUNT(*), COUNT(DISTINCT shape_id)) AS avg_edges_per_shape
FROM `{project_id}.{dataset_id}.shape_edges`
WHERE shape_id IN (
  SELECT DISTINCT shape_id FROM `{project_id}.{dataset_id}.stg_trips` WHERE route_id = @route_id
)
"""

params = [bigquery.ScalarQueryParameter("route_id", "STRING", route_id)]
for label, q in [
    ("trip_edges overview", q_overview),
    ("edges per trip stats", q_edges_per_trip_stats),
    ("shape_edges stats (if table exists)", q_shape_edges_stats),
]:
    try:
        job = client.query(q, job_config=bigquery.QueryJobConfig(query_parameters=params), location=location)
        df = job.result().to_dataframe()
        print(label)
        display(df)
    except Exception as e:
        print(f"{label} failed: {e}")

## Alternative path: materialize shape_edges then expand to trip_edges
This avoids recomputing spatial intersections per trip when many trips share the same shape_id. Steps:
1) Build `shape_edges` once
2) Insert `trip_edges` for a route from `shape_edges` (fast expansion)

## Incremental builder: shape_edges one shape at a time
Use this when you hit CPU limits or want to resume partially. It inserts shape→edge links for each shape_id with an anti-join to skip already-processed pairs.

In [21]:
# Build shape_edges incrementally, one shape at a time (filtered by current route)
shape_edges_by_shape_path = sql_dir / "shape_edges_insert_by_shape.sql"

# Ensure the base table exists (empty create if missing)
create_if_missing = f"""
CREATE SCHEMA IF NOT EXISTS `{project_id}.{dataset_id}`;
CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.shape_edges` (
  shape_id STRING,
  edge_id STRING,
  road_name STRING,
  highway STRING,
  oneway STRING,
  maxspeed STRING,
  edge_length_m FLOAT64,
  overlap_m FLOAT64,
  edge_geom GEOGRAPHY
) CLUSTER BY shape_id, edge_id;
"""
_ = run_sql_loc(create_if_missing)

# Fetch shapes for the current route
route_shapes = fetch_shape_ids_for_route(route_id)
print(f"Route {route_id} has {len(route_shapes)} shape_ids to process")

# Process one shape per query; you can adjust to batches if desired
for idx, sid in enumerate(route_shapes, start=1):
    sql = render_sql(load_sql(str(shape_edges_by_shape_path)), {"project_id": project_id, "dataset_id": dataset_id, "shape_id": sid})
    print(f"[{idx}/{len(route_shapes)}] Inserting shape_edges for shape_id={sid}")
    _ = run_sql_loc(sql)

print("Done building shape_edges incrementally for route", route_id)

Job 8728a49a-8b76-4903-bbf6-a4ed641a13f6 done. location=australia-southeast1 bytes=0
Route INN-202 has 16 shape_ids to process
[1/16] Inserting shape_edges for shape_id=1286-79821-0d6c86f9
Route INN-202 has 16 shape_ids to process
[1/16] Inserting shape_edges for shape_id=1286-79821-0d6c86f9


BadRequest: 400 No matching signature for operator = for argument types: STRING, INT64
  Signature: T1 = T1
    Unable to find common supertype for templated argument <T1>
      Input types for <T1>: {INT64, STRING} at [41:35]; reason: invalidQuery, location: query, message: No matching signature for operator = for argument types: STRING, INT64
  Signature: T1 = T1
    Unable to find common supertype for templated argument <T1>
      Input types for <T1>: {INT64, STRING} at [41:35]

Location: australia-southeast1
Job ID: 396ee8b2-3610-4763-a7f3-7c3f8a00627f


In [ ]:
# Run alternative path: shape_edges -> trip_edges (by route)
shape_edges_path = sql_dir / "shape_edges.sql"
trip_edges_from_shapes_path = sql_dir / "trip_edges_insert_by_route_from_shapes.sql"
print(shape_edges_path)
print(trip_edges_from_shapes_path)
shape_edges_sql = render_sql(load_sql(str(shape_edges_path)), {"project_id": project_id, "dataset_id": dataset_id})
_ = run_sql_loc(shape_edges_sql)
trip_from_shapes_sql = render_sql(load_sql(str(trip_edges_from_shapes_path)), {"project_id": project_id, "dataset_id": dataset_id, "route_id": route_id})
_ = run_sql_loc(trip_from_shapes_sql)

## Speed-ups and options
- Prefer the shape_edges -> trip_edges alternative path (compute spatial joins once per shape).
- Run shapes in parallel (careful with quotas): 2–4 workers is a safe start.
- Reduce geometry costs: smaller buffer (e.g., 10m), coarser simplify (e.g., 7–10), higher overlap threshold (e.g., 30–50m).
- Use smaller batches if you see CPU ratio errors; anti-join prevents duplicates on reruns.

In [ ]:
# Optional: run shape chunks in parallel (advanced)
from concurrent.futures import ThreadPoolExecutor, as_completed
max_workers = 3  # start with 2-4; increase cautiously
one_shape_at_a_time = True
chunk_size = 1 if one_shape_at_a_time else 5
all_shapes = fetch_shape_ids_for_route(route_id)
params = {"project_id": project_id, "dataset_id": dataset_id, "route_id": route_id}
base_sql = render_sql(load_sql(str(trip_edges_insert_by_route_path)), params)

def exec_batch(batch: list[str]) -> tuple[int, int]:
    sql = render_insert_with_shape_filter_and_antijoin(base_sql, batch, route_id, project_id, dataset_id)
    run_sql_loc(sql)
    return (len(batch), 1)

batches = [all_shapes[i:i+chunk_size] for i in range(0, len(all_shapes), chunk_size)]
submitted = 0
with ThreadPoolExecutor(max_workers=max_workers) as ex:
    futures = {ex.submit(exec_batch, b): b for b in batches}
    for fut in as_completed(futures):
        batch = futures[fut]
        try:
            cnt, _ = fut.result()
            print(f"Done batch of {cnt} shapes: {batch}")
        except Exception as e:
            print(f"Batch failed for {batch}: {e}")
        submitted += 1
print(f"Completed {submitted}/{len(batches)} batches")